# Challenge Code

### Basic data extraction

In [1]:
import numpy as np
import pandas as pd

from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
import lime
import lime.lime_tabular

In [2]:
#df = pd.read_csv("xML Challenge Dataset and Data Dictionary/heloc_dataset_v1.csv")
#df = pd.read_csv("new_data_set.csv")
df = pd.read_csv("Processed Data/data_mean3.csv")
feature_names = list(df)
data = df.values

np

y = data[:,:1]
X = data[:,1:]

### Data Manipulation and Processing

In [3]:
# Getting rid of the -7, -8, -9 values
#np.place(X, X < 0, 0)

print(X.shape)
print(y.shape)

# Removing all samples with any negative values
# row_no = 0 
# for row in X:
#     for col in row:
#         if (col < 0):
#             X = np.delete(X, row_no, 0)
#             y = np.delete(y, row_no, 0)
#             row_no -= 1
#             break
            
#     row_no += 1
# print(X.shape)

(9871, 24)
(9871, 1)


In [4]:
# Removing the two contentious columns
# X = np.delete(X, 9, 1)
# X = np.delete(X, 10, 1)
# X.shape

In [5]:
#X_scaled = X
#X_scaled1 = (X-X.mean())/(X.std())
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
num_samples , num_attributes = X_scaled.shape

#print("num_samples = " + str(num_samples) + ' , ' + "num_attributes = " + str(num_attributes))

In [6]:
X_tr = X_scaled[:int(0.5*num_samples)]
X_val = X_scaled[int(0.5*num_samples):int(0.75*num_samples)]
X_test = X_scaled[int(0.75*num_samples):]

y_tr = y[:int(0.5*num_samples)]
y_val = y[int(0.5*num_samples):int(0.75*num_samples)]
y_test = y[int(0.75*num_samples):]

print("Training samples: ", X_tr.shape[0])
print("Validation samples: ", X_val.shape[0])
print("Test samples: ", X_test.shape[0])

Training samples:  4935
Validation samples:  2468
Test samples:  2468


### Visualization

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [ ]:
pca = PCA(n_components=3,svd_solver='full')
pca.fit(X_scaled)
X_pca = pca.transform(X_scaled)

total=0
for ev in pca.explained_variance_ratio_:
    total += ev

print("PCA Variance Explained:",total*100, "%")

In [ ]:
model = TSNE(n_components=3)
# tsne.fit(X_scaled)
# X_tsne = tsne.transform(X_scaled)


In [ ]:
X_tsne = model.fit_transform(X_scaled)

In [ ]:
print("TSNE KL Divergence:",model.kl_divergence_)

In [ ]:
import copy
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

# y_binary = copy.deepcopy(y)
# np.place(y_binary, y_binary == "Bad", 0)
# np.place(y_binary, y_binary == "Good", 1)

# y_binary = y_binary.reshape(y_binary.shape[0],)

fig = plt.figure()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
#ax.scatter(X_pca.transpose()[0], X_pca.transpose()[1], X_pca.transpose()[2],c=y_binary,cmap="flag",label='Testing')
ax.scatter(X_tsne.transpose()[0], X_tsne.transpose()[1], X_tsne.transpose()[2],c=y,cmap="flag",label='Testing')

ax.legend()
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(X_tsne.transpose()[0],X_tsne.transpose()[1],c=y)
ax.grid()
plt.show()

### Logistic Regression

In [ ]:
logreg = linear_model.LogisticRegression(penalty="l2",C=0.0015, solver="liblinear")
# Fit the model on the training set.
logreg.fit(X_tr, y_tr.reshape(y_tr.shape[0],))

pred = logreg.predict(X_val)
acc = np.mean(pred.reshape(pred.shape[0],1) == y_val)
coefs = logreg.coef_
print("Accuracy:", acc*100, "%")

# print(abs(coefs))



In [ ]:
# ranks = []
# # coef_sort = sorted(abs(coefs),reverse=True)
# coefs = abs(coefs.reshape(coefs.shape[1],))
# coef_sort = coefs
# coef_sort.sort()

# print(coef_sort)
# r = 0
# for weight in coefs:
#     print(coef_sort.index(weight))
    

    

In [ ]:
# C_acc = []
# C_options = [0.0000001,0.000001,0.00001,0.0001,0.001,0.01,0.1,1,10]

# for test in C_options:
#     C_acc.append(acc_test_log(test))


### LIME implementation

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(X_tr ,feature_names = feature_names, 
                                                   discretize_continuous=True)
exp = explainer.explain_instance(X_test[2], logreg.predict_proba, num_features=23)

In [ ]:
exp.show_in_notebook(show_table=True, show_all=False)

### Support Vector Machines - Linear Kernel

In [7]:
from sklearn import svm

In [8]:
def acc_test_SVM(C_val):
    print("Running ", C_val)
    svc = svm.SVC(kernel='linear',C=C_val)
    svc.fit(X_tr,y_tr.reshape(y_tr.shape[0],))
    y_pred = svc.predict(X_val)
    acc = np.mean(y_pred.reshape(y_pred.shape[0],1) == y_val)
    return acc

In [9]:
C_acc = []
C_options = [0.00001,0.0001,0.001,0.01,0.1,1,10]

for test in C_options:
    C_acc.append(acc_test_SVM(test))

Running  1e-05


KeyboardInterrupt: 

In [ ]:
plt.semilogx(C_options,C_acc,'.-',color='blue')
plt.title("Accuracy depending on C value")
plt.xlabel('C value')
plt.ylabel('Test Accuracy')
plt.grid()

In [ ]:
svm_lin = svm.SVC(kernel='linear',C=0.0001)
svm_rbf = svm.SVC(kernel='rbf',C=0.0001)
svm_pol = svm.SVC(kernel='poly',C=0.0001,degree=2)

svm_lin.fit(X_tr,y_tr.reshape(y_tr.shape[0],))
svm_rbf.fit(X_tr,y_tr.reshape(y_tr.shape[0],))
svm_pol.fit(X_tr,y_tr.reshape(y_tr.shape[0],))

y_lin = svm_lin.predict(X_val)
y_rbf = svm_rbf.predict(X_val) 
y_pol = svm_pol.predict(X_val) 

In [ ]:
acc_lin = np.mean(y_lin.reshape(y_lin.shape[0],1) == y_val)
acc_rbf = np.mean(y_rbf.reshape(y_rbf.shape[0],1) == y_val)
acc_pol = np.mean(y_pol.reshape(y_pol.shape[0],1) == y_val)
print("Linear Kernel Accuracy:", acc_lin*100, "%")
print("RBF Kernel Accuracy:", acc_rbf*100, "%")
print("Polynomial Kernel Accuracy:", acc_pol*100, "%")